# <center> Практика. EDA + Feature Engineering. Соревнование на Kaggle </center>

### **Постановка задачи:**

**ИССЛЕДОВАНИЕ ДАННЫХ:**

✍ В этом модуле мы будем работать с датасетом, в котором содержатся сведения о 515 000 отзывов на отели Европы. Модель, которую мы будем обучать, должна предсказывать рейтинг отеля по данным сайта Booking на основе имеющихся в датасете данных.

***ДАННЫЕ САЙТА BOOKING:***

Наименование столбца:  | Описание столбца:
------- | --------
***hotel_address***   | адрес отеля;
***review_date***   | дата, когда рецензент разместил соответствующий отзыв;
***average_score***   | средний балл отеля, рассчитанный на основе последнего комментария за последний год;
***hotel_name***   | название отеля;
***reviewer_nationality***   | страна рецензента;
***negative_review***   | отрицательный отзыв, который рецензент дал отелю;
***review_total_negative_word_counts***   | общее количество слов в отрицательном отзыв;
***positive_review***   | положительный отзыв, который рецензент дал отелю;
***review_total_positive_word_counts***   | общее количество слов в положительном отзыве.
***reviewer_score***   | оценка, которую рецензент поставил отелю на основе своего опыта;
***total_number_of_reviews_reviewer_has_given***   | количество отзывов, которые рецензенты дали в прошлом;
***total_number_of_reviews***   | общее количество действительных отзывов об отеле;
***tags***   | теги, которые рецензент дал отелю;
***days_since_review***   | количество дней между датой проверки и датой очистки;
***additional_number_of_scoring***   | есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
***lat***   | географическая широта отеля;
***lng***   | географическая долгота отеля.

***Содержание работы:***

1. Импорт библиотек и загрузка данных.
2. Очистка данных.
3. Исследование данных.
4. Генерация признаков.
5. Преобразование признаков.
6. Отбор признаков.
7. Обучение модели.

## **1. Импорт библиотек и загрузка данных:**

In [1]:
# Загружаем необходимые библиотеки:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings("ignore")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [4]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/sf-booking//hotels_train.csv'

In [ ]:
df_train.info()

In [ ]:
df_train.head(2)

In [ ]:
df_test.info()

In [ ]:
df_test.head(2)

In [ ]:
sample_submission.head(2)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем
# трейн и тест в один датасет:

df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score,
# мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True)
# объединяем

In [ ]:
data.info()

In [ ]:
data['lat'] = data['lat'].fillna(0, inplace=True)
data['lng'] = data['lng'].fillna(0, inplace=True)

In [ ]:
data.nunique(dropna=False)

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True)

### ***Вывод:*** в датасете 515738 записей (есть пропущенные значения).

## **2. Очистка данных:**

2.1. Проверим данные на наличие дубликатов и удалим найденные дубликаты:

In [ ]:
print('В датасете в тренировочной выборке: {} дубликатов.'.format(df_train[df_train.duplicated()].shape[0]))
print('В датасете в тестовой выборке: {} дубликатов.'.format(df_test[df_test.duplicated()].shape[0]))

df_train.drop_duplicates(inplace=True)
print('Количество строк после удаления дубликатов в тренировочной выборке составляет: {}.'.format(df_train.shape[0]))

### ***Вывод:*** найдены дубликаты в обеих частях датасета мы проведем удаление дубликатов, только в тренировочной выборке (представленные данные содержат 307 дубликатов), т.к. тестовая выборка фиксирована и ее изменять нельзя по правилам соревнования на kaggle. Количество строк после удаления дубликатов в тренировочной выборке составляет 386496.

## **3. Исследование данных:**

3.1. Классифицируем все признаки на числовые и категориальные:

***Категориальные признаки:***
1. hotel_address
2. review_date
3. hotel_name
4. reviewer_nationality
5. negative_review
6. positive_review
7. tags
8. days_since_review

***Числовые признаки:***
1. additional_number_of_scoring
2. average_score
3. review_total_negative_word_counts
4. total_number_of_reviews
5. review_total_positive_word_counts
6. total_number_of_reviews_reviewer_has_given
7. lat
8. lng
9. sample
10. reviewer_score

3.2. Выведем на экран основные статистические характеристики
данных по каждому числовому признаку:

In [ ]:
data.describe(include = 'all')

### **Вывод:** с помощью метода describe() определили основные статистические характеристики для каждого из признаков.

## **4. Генерация признаков:**

4.1. Иследуем признак hotel_address и создадим новые признаки ***'country'*** - страна, ***'city'*** - город, ***'hotel_city_count'*** - количество отелей в городе:

In [ ]:
# Создание нового признака 'country':

data['country'] = data['hotel_address'].apply(lambda x: x.split()[-1] 
        if x.split()[-1] != 'Kingdom' 
        else ' '.join(x.split()[-2:]))

print('\n В датасете представлены отели из '+ str(
    data['country'].nunique()) + ' стран:\n')
print(data['country'].value_counts())

In [ ]:
# Создание нового признака 'city':

data['city'] = data.apply(lambda x: x['hotel_address'].split()[-5] 
        if x['country'] == 'United Kingdom'
        else x['hotel_address'].split()[-2], axis=1)

print('\n В датасете представлены отели из '+ str(
    data['city'].nunique()) + ' городов:\n')
print(data['city'].value_counts())

In [ ]:
# Создание нового числового признака 'hotel_city_count':

data['hotel_city_count'] = data.groupby(
    'city')['hotel_name'].transform('count')

In [ ]:
# Кодируем признаки методом OneHotEncoding:

data = pd.get_dummies(data, columns = ['country'])
data = pd.get_dummies(data, columns = ['city'])

4.2. Иследуем признак 'reviewer_nationality':

In [ ]:
# Выделим десять лидирующих национальностей:

nationality10 = data['reviewer_nationality'].value_counts().nlargest(10)

data['reviewer_nationality'] = data['reviewer_nationality'].apply(
    lambda x: x if x in nationality10 else 'other')

fig = plt.figure(figsize = (16,10))
barplot = sns.boxplot(
    data = data, x = 'reviewer_score',
    y = 'reviewer_nationality', orient = 'h')

In [ ]:
# Кодируем признаки методом OneHotEncoding:

data = pd.get_dummies(data, columns = ['reviewer_nationality'])

4.3. Иследуем признак 'review_date'. Приведем дату отзыва к формату datetime.
Создадим новые числовые признаки:

***'review_year'*** - год;  
***'review_month'*** - месяц;  
***'review_quarter'*** - квартал;  
***'review_season'*** - сезон, в котором рецензент разместил соответствующий отзыв

In [ ]:
# Приведем дату отзыва к формату datetime:

data['review_date'] = pd.to_datetime(data['review_date']).dt.date

# Выделим год, месяц, квартал из даты отзыва:

data['review_year'] = pd.to_datetime(data['review_date']).dt.year
data['review_month'] = pd.to_datetime(data['review_date']).dt.month
data['review_quarter'] = pd.to_datetime(data['review_date']).dt.quarter

# Создание нового признака 'review_season':

data['review_season'] = data['review_month']

def get_season(month):
        
    if month in list(range(3,6)):
        return 'spring'
    
    if month in list(range(6,9)):
        return 'summer'
    
    if month in list(range(9,12)):
        return 'autumn'
    
    else:
        return 'winter'

data['review_season'] = data['review_season'].apply(get_season)

# Кодируем признак методом OneHotEncoding:

data = pd.get_dummies(data, columns=['review_season'])

In [ ]:
# Выделим сезонность оценок:

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

sns.countplot(data['review_month'], ax=axes[0]);
axes[0].set(xlabel='Месяц:', ylabel='Количество записей:')
axes[0].set_title('Распределение данных по месяцам:')

sns.countplot(data['review_quarter'],  ax=axes[1]);
axes[1].set(xlabel='Квартал:', ylabel='Количество записей:')
axes[1].set_title('Распределение данных по кварталам:')
fig.show()

4.4 Иследуем признак 'tags' и создадим новый числовой признак ***'nights'*** - количество ночей в отеле:

In [ ]:
data['tags'] = data['tags'].apply(
    lambda x: x.replace("[' ", "").replace(" ']", "").split(" ', ' "))

def get_night(arg):
    
    for tag in arg:
        if 'Stayed' in tag:
            return int(tag.split()[1])
            
# Создание нового признака 'nights':
data['nights'] = data['tags'].apply(get_night)

# Заменим пропуски в признаке медианой:
data['nights'] = data['nights'].fillna(data.nights.median())

4.5 Иследуем признак 'tags' и создадим новый числовой признак ***'type_trip'*** - тип поездки:

In [ ]:
# Создание нового признака 'type_trip':

data['type_trip'] = data['tags'].apply(
    lambda x: 1 if 'Business' in x else 0)

4.6 Иследуем признак 'tags' и создадим новый признак ***'type_of_travelers'*** - тип путешественников:

In [ ]:
def set_why_stayed(value):
    
    if 'Group' in  value:
        return 'Group'
    if 'Couple' in value:
        return 'Couple'  
    if 'Solo traveler' in value:
        return 'Solo traveler'
    if 'Family with young children' in value:
        return 'Family with young children'
    if 'Family with older children' in value:
        return 'Family with older children' 
    return np.nan

# Создание нового признака 'type_of_travelers':

data['type_of_travelers'] = data['tags'].apply(set_why_stayed)
data['type_of_travelers'] = data['type_of_travelers'].fillna(
    data['type_of_travelers'].mode().iat[0])

# Кодируем признак методом OneHotEncoding:

data = pd.get_dummies(data, columns=['type_of_travelers'])

4.7 Иследуем признак 'tags' и создадим новый признак ***'type_of_apartment'*** - тип проживания:

In [ ]:
def get_room(arg):
    
    for tag in arg:
        if 'Room' in tag:
            return tag.strip()
    return 'Unknown'
            
# Создание нового признака 'type_of_apartment':

data['type_of_apartment'] = data['tags'].apply(get_room)

4.8 Иследуем признак 'days_since_review' выделим числа:

In [ ]:
data['days_since_review'] = data['days_since_review'].apply(
    lambda x: int(x.split()[0]))

4.9 Иследуем признак 'negative_review' и создадим числовой признак ***'is_negative'*** - отрицательные отзывы, которые не имеют отрицательного значения:

In [ ]:
# Создание нового признака 'is_negative':

data['is_negative'] = data['negative_review'].apply(
    lambda x: 0 if x == 'No Negative' else 1)

4.10 Иследуем признак 'positive_review' и создадим числовой признак ***'is_positive'*** - положительные отзывы, которые не имеют положительного значения:

In [ ]:
# Создание нового признака 'is_positive':

data['is_positive'] = data['positive_review'].apply(
    lambda x: 0 if x =='No Positive' else 1)

## **6. Отбор признаков:**

6.1. Определим в данных неинформативные признаки, которые не будут участвовать в исследовании:

In [ ]:
# Удаляем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет,
# просто выберим их и удалим:

object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

In [ ]:
data.info()

In [ ]:
# Теперь выделим тестовую часть:

train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

## **7. Обучение модели**

In [ ]:
# Воспользуемся специальной функцие train_test_split
# для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size):

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# Проверяем:

test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
# инструмент для создания и обучения модели:

from sklearn.ensemble import RandomForestRegressor

# инструменты для оценки точности модели:

from sklearn import metrics

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ):

model = RandomForestRegressor(
    n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных:

model.fit(X_train, y_train)

# Используем обученную модель для предсказания
# рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred:

y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test),
# и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает
# среднее отклонение предсказанных значений от фактических.

print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести
# самые важные признаки для модели:

plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)